In [ ]:
!pip install -q tensorflow datasets transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dnepozitek/polyvore-outfits")

print("Path to dataset files:", path)

100%|██████████| 6.21G/6.21G [04:39<00:00, 23.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dnepozitek/polyvore-outfits/versions/1


In [ ]:
import os
import json
import pandas as pd

polyvore_dir = 'polyvore_outfits'
filename = os.path.join(polyvore_dir, 'polyvore_item_metadata.json')

In [ ]:
with open(filename,'r') as fl:
    data = json.load(fl)

df = pd.DataFrame(data).T

df['id'] = df.index
df['path'] = df['id'].apply(lambda x: os.path.join(polyvore_dir, 'images', x + '.jpg'))
df['label'] = df['category_id']

df.head()
df[['id', 'path', 'label']].to_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata.csv'),index=False)

In [ ]:
from collections import defaultdict
import random

def split_data_with_limit(df, k=100, train_ratio=0.8, seed=42):
    """
    Splits the data into train and test sets with a limit of k images per category.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'category_id' column.
        k (int): Maximum number of images per category in both train and test sets.
        train_ratio (float): Proportion of data to allocate to the train set.
        seed (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame, pd.DataFrame: Train and test DataFrames.
    """
    # Group data by category_id
    grouped = defaultdict(list)
    for _, row in df.iterrows():
        grouped[row['label']].append(row)

    train_data = []
    test_data = []
    random.seed(seed)  # Set random seed for reproducibility

    for category, items in grouped.items():
        # Shuffle the items within each category
        random.shuffle(items)

        # Limit to k images per category
        items = items[:k]

        # Split into train and test sets
        split_idx = int(len(items) * train_ratio)
        train_data.extend(items[:split_idx])
        test_data.extend(items[split_idx:])

    # Convert back to DataFrame
    train_df = pd.DataFrame(train_data)
    test_df = pd.DataFrame(test_data)

    return train_df, test_df

# Split the data with a limit of 100 images per category
train_df, test_df = split_data_with_limit(df, k=100, train_ratio=0.8)

# Print summary
print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

# Verify category distributions
print("Train set distribution:")
print(train_df['label'].value_counts())
print("\nTest set distribution:")
print(test_df['label'].value_counts())

# Save to CSV
train_df.to_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata_train.csv'),index=False)
test_df.to_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata_test.csv'),index=False)

Train set size: 8845
Test set size: 2239
Train set distribution:
label
15      80
4496    80
51      80
10      80
310     80
        ..
306      4
269      4
289      4
4468     3
279      1
Name: count, Length: 153, dtype: int64

Test set distribution:
label
15      20
28      20
231     20
51      20
10      20
        ..
4473     1
279      1
306      1
289      1
4468     1
Name: count, Length: 153, dtype: int64


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

df = pd.read_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata.csv'))
train_df = pd.read_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata_train.csv'))
test_df = pd.read_csv(os.path.join(polyvore_dir, 'polyvore_item_metadata_test.csv'))

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocessing function
def preprocess_data(df, image_dir, target_size=(224, 224), batch_size=128):
    datagen = ImageDataGenerator(
        rescale=1./255,  # Normalize pixel values
        # horizontal_flip=True,  # Data augmentation
        validation_split=0.01  # Split training into train/validation
    )

    train_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col="path",  # Column with image filenames
        y_col="label",  # Column with labels
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",  # One-hot encode labels
        subset="training"  # Use training subset
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=df,
        directory=image_dir,
        x_col="path",
        y_col="label",
        target_size=target_size,
        batch_size=batch_size,
        class_mode="categorical",
        subset="validation"  # Use validation subset
    )

    return train_generator, val_generator

# Define paths and preprocess the data
# training_df = train_df
# train_df['label'] = train_df['label'].astype(str)

training_df = df[~df.id.isin(test_df.id)]
training_df['label'] = training_df['label'].astype(str)

image_dir = '.'  # Path to the image directory
train_generator, val_generator = preprocess_data(training_df, image_dir, batch_size=128*2)

<ipython-input-31-976171bcae79>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['label'] = training_df['label'].astype(str)


Found 246282 validated image filenames belonging to 153 classes.
Found 2487 validated image filenames belonging to 153 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = tf.keras.applications.EfficientNetB2( include_top=False, weights='imagenet')


# Freeze the base model
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dropout(0.5)(x)  # Add Dropout for regularization
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Number of layers in the base model:  175


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    # validation_data=val_generator,
    epochs=5,  # Number of epochs
    # steps_per_epoch=train_generator.samples // train_generator.batch_size,
    # validation_steps=val_generator.samples // val_generator.batch_size
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


963/963 ━━━━━━━━━━━━━━━━━━━━ 1204s 1s/step - accuracy: 0.1708 - loss: 3.6498
Epoch 2/5
963/963 ━━━━━━━━━━━━━━━━━━━━ 1131s 1s/step - accuracy: 0.4491 - loss: 2.0079
Epoch 3/5
963/963 ━━━━━━━━━━━━━━━━━━━━ 1130s 1s/step - accuracy: 0.5174 - loss: 1.7108
Epoch 4/5
963/963 ━━━━━━━━━━━━━━━━━━━━ 1129s 1s/step - accuracy: 0.5513 - loss: 1.5656
Epoch 5/5
963/963 ━━━━━━━━━━━━━━━━━━━━ 1126s 1s/step - accuracy: 0.5724 - loss: 1.4758


In [ ]:
model.save('polyvore_resnet.h5')